In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm.laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's submodule for Laplace kernel."

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
max_level = 4
fmm.configure(p, ncrit, max_level)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
skip_P2P = True   # flag to skip near-field evaluation
nodes = fmm.build_list(skip_P2P)

#### check colleagues of a node (after creating the interaction list)

In [9]:
print(len(nodes))
print(type(nodes[500].colleagues))   # list of nodes

4681
<class 'list'>


In [10]:
print(nodes[500].x)

for i, colleague in enumerate(nodes[500].colleagues):
    print(i, colleague.x, colleague.level, colleague.key)

[0.218747, 0.343750, 0.406248]
0 [0.156247, 0.281250, 0.343747] 4 810
1 [0.218747, 0.281250, 0.343747] 4 814
2 [0.281247, 0.281250, 0.343747] 4 1034
3 [0.156247, 0.343750, 0.343747] 4 812
4 [0.218747, 0.343750, 0.343747] 4 816
5 [0.281247, 0.343750, 0.343747] 4 1036
6 [0.156247, 0.406250, 0.343747] 4 826
7 [0.218747, 0.406250, 0.343747] 4 830
8 [0.281247, 0.406250, 0.343747] 4 1050
9 [0.156247, 0.281250, 0.406248] 4 817
10 [0.218747, 0.281250, 0.406248] 4 821
11 [0.281247, 0.281250, 0.406248] 4 1041
12 [0.156247, 0.343750, 0.406248] 4 819
13 [0.218747, 0.343750, 0.406248] 4 823
14 [0.281247, 0.343750, 0.406248] 4 1043
15 [0.156247, 0.406250, 0.406248] 4 833
16 [0.218747, 0.406250, 0.406248] 4 837
17 [0.281247, 0.406250, 0.406248] 4 1057
18 [0.156247, 0.281250, 0.468748] 4 818
19 [0.218747, 0.281250, 0.468748] 4 822
20 [0.281247, 0.281250, 0.468748] 4 1042
21 [0.156247, 0.343750, 0.468748] 4 820
22 [0.218747, 0.343750, 0.468748] 4 824
23 [0.281247, 0.343750, 0.468748] 4 1044
24 [0.15624

Some nodes may not have all 27 colleagues. In that case, `len(colleagues)` is still 27, but the missing colleagues are filled with `None`. For example, the level 1 node `nodes[2]` only have 8 colleagues, other entries are `None`:

In [11]:
print(nodes[2].colleagues)

[None, None, None, None, None, None, None, None, None, None, None, None, <exafmm.laplace.Node object at 0x7fe16805c3b0>, <exafmm.laplace.Node object at 0x7fe145243d18>, None, <exafmm.laplace.Node object at 0x7fe145243d88>, <exafmm.laplace.Node object at 0x7fe145243dc0>, None, None, None, None, <exafmm.laplace.Node object at 0x7fe145243df8>, <exafmm.laplace.Node object at 0x7fe145243e30>, None, <exafmm.laplace.Node object at 0x7fe145243ce0>, <exafmm.laplace.Node object at 0x7fe145243ea0>, None]


#### precompute

In [12]:
fmm.precompute()

#### evaluate potentials

In [13]:
potentials = fmm.evaluate()

P2M                  : 8.5529000e-02
M2M                  : 1.0778600e-01
P2L                  : 4.0300000e-04
M2P                  : 1.8610000e-03
P2P                  : 3.3400000e-04
M2L                  : 5.8869200e-01
L2L                  : 9.9823000e-02
L2P                  : 9.3295000e-02


In [14]:
potentials[::10000]

array([5954.45925253, 6991.37239948, 6132.82356837, 6518.49023592,
       8017.81772947, 8772.72503922, 7245.22883524, 7565.21061384,
       7461.32977143, 7654.77503667])

#### check accuracy

For now, this is only valid when skip_P2P is disabled.

In [15]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 3.6760395e-02
Gradient Error       : 1.4845553e-01


#### update charges and run FMM iteratively

In [ ]:
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy